# Данные

In [1]:
!wget https://www.dropbox.com/s/nd7v1fod89xla6j/vk_texts_with_sources.csv

--2019-05-29 21:13:21--  https://www.dropbox.com/s/nd7v1fod89xla6j/vk_texts_with_sources.csv
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1, 2620:100:6026:1::a27d:4601
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/nd7v1fod89xla6j/vk_texts_with_sources.csv [переход]
--2019-05-29 21:13:22--  https://www.dropbox.com/s/raw/nd7v1fod89xla6j/vk_texts_with_sources.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uca976fd562c538cb46e54f99031.dl.dropboxusercontent.com/cd/0/inline/Ah21DsxdPCpDGcRvEmjX7WQhM6saL_c2Hbk2FS-eJktwRFaC3pO4Ugs0w8zsWnV8nzyqCKxG6zi0TGAsCKa83DseAvEmLF7cf0e1nBU_g-llhA/file# [переход]
--2019-05-29 21:13:23--  https://uca976fd562c538cb46e54f99031.dl.dropboxusercontent.com/cd/0/inline/Ah21DsxdPCpDGcRvEmjX7WQhM6saL_c2Hbk2FS-eJktwRFaC3pO4Ugs0w8zsWnV8nzyqCKxG6zi0TGAsCKa

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv('vk_texts_with_sources.csv', usecols = ['text', 'source'])
df.head()

text source
0  Начальник Главного оперативного управления Ген...    mil
1  Артиллерийские подразделения общевойскового об...    mil
2  Подразделения морской пехоты Каспийской флотил...    mil
3  Команды на всеармейских этапах конкурсов АрМИ-...    mil
4  На большом учебно-методическом командирском сб...    mil

# Домашнее задание

В этом домашнем задании вы будете решать задачу тематической классификации. Даны тексты, опубликованные в нескольких пабликах VK.com, посвященных государственным и муниципальным службам. Формально задача заключается в том, чтобы по тексту ($d$) определить в каком паблике он опубликован, то есть, к какому классу $c$ он принадлежит. 

## Задание 1 [1 балл]. Описательные статистики
Посчитайте:
* количество текстов и количество классов
* количество слов (без лемматизации и с лемматизацией) в коллекции
* среднюю длину текста в словах и символах
* найдите 5 самых частых существительных в текстах каждого паблика 

*Рекомендуем использовать pandas для расчета описательных статистик.*

In [8]:
df.describe(include=[np.object])

text   source
count                       11526    11625
unique                      11149        4
top     #СоветМЧС #МЧС #МЧСРОССИИ  mchsgov
freq                           45     3030

У нас 4 класса и 11625 различных текстов.

In [36]:
import nltk
import re
from pymystem3 import Mystem

nltk.download('punkt')
mystem_analyzer = Mystem()

lemmatized = []
unlemmatized = []

for doc in df['text']:
    text = re.sub("<br>", "", str(doc))
    word_list = nltk.word_tokenize(text)
    unlemmatized.append(word_list)
    lemmatized.append([mystem_analyzer.lemmatize(word)[0] for word in word_list])

[nltk_data] Downloading package punkt to /home/valeria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
print(unlemmatized[0][:10])
print("--------")
print(lemmatized[0][:10])

['Начальник', 'Главного', 'оперативного', 'управления', 'Генерального', 'штаба', 'Вооруженных', 'Сил', 'РФ', 'генерал-полковник']
--------
['начальник', 'главный', 'оперативный', 'управление', 'генеральный', 'штаб', 'вооруженный', 'сила', 'рф', 'генерал-полковник']


In [50]:
from itertools import chain

count_unlem = 0

for el in unlemmatized:
    count_unlem += len(el)

cont_lemmatized = list(chain.from_iterable(lemmatized))

print("Количество токенов в датасете:", count_unlem)
print("Количество уникальных лемм:", len(set(cont_lemmatized)))

Количество токенов в датасете: 1163263
Количество уникальных лемм: 33153


In [41]:
words_per_doc = np.mean([len(text) for text in unlemmatized])
symbols_per_doc = np.mean([len(str(doc)) for doc in df["text"]])

print("Среднее количество слов на документ:", int(words_per_doc))
print("Среднее количество символов на документ:", int(symbols_per_doc))

Среднее количество слов на документ: 100
Среднее количество символов на документ: 684


In [105]:
from collections import Counter
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm

morph = MorphAnalyzer()

tops = Counter()

for i in tqdm(range(len(df['source']))):
    for word in range(len(unlemmatized[i])):
        if morph.tag(unlemmatized[i][word])[0].POS == 'NOUN':
            tops.update([(unlemmatized[i][word], df["source"][i])])










  0%|          | 0/11625 [00:00<?, ?it/s]








  0%|          | 1/11625 [00:00<1:19:44,  2.43it/s]








  0%|          | 5/11625 [00:00<57:28,  3.37it/s]  








  0%|          | 8/11625 [00:00<42:26,  4.56it/s]








  0%|          | 12/11625 [00:00<31:16,  6.19it/s]








  0%|          | 15/11625 [00:00<24:31,  7.89it/s]








  0%|          | 18/11625 [00:01<19:27,  9.94it/s]








  0%|          | 23/11625 [00:01<14:55, 12.95it/s]








  0%|          | 27/11625 [00:01<12:00, 16.10it/s]








  0%|          | 31/11625 [00:01<11:40, 16.55it/s]








  0%|          | 34/11625 [00:01<10:12, 18.93it/s]








  0%|          | 38/11625 [00:01<09:16, 20.82it/s]








  0%|          | 41/11625 [00:01<11:21, 16.99it/s]








  0%|          | 44/11625 [00:02<10:12, 18.90it/s]








  1%|          | 62/11625 [00:02<07:33, 25.51it/s]








  1%|          | 70/11625 [00:02<06:00, 32.03it/s]








  1%|          | 77/11625 [00:02<05:03, 38.08it/s]






  4%|▍         | 495/11625 [00:15<06:51, 27.05it/s]








  4%|▍         | 502/11625 [00:15<05:41, 32.58it/s]








  4%|▍         | 507/11625 [00:15<05:30, 33.63it/s]








  4%|▍         | 512/11625 [00:15<05:29, 33.69it/s]








  4%|▍         | 516/11625 [00:16<07:32, 24.54it/s]








  4%|▍         | 523/11625 [00:16<06:08, 30.17it/s]








  5%|▍         | 528/11625 [00:16<06:18, 29.34it/s]








  5%|▍         | 532/11625 [00:16<07:51, 23.53it/s]








  5%|▍         | 536/11625 [00:16<07:00, 26.34it/s]








  5%|▍         | 542/11625 [00:16<05:51, 31.58it/s]








  5%|▍         | 553/11625 [00:17<04:46, 38.59it/s]








  5%|▍         | 559/11625 [00:17<04:22, 42.13it/s]








  5%|▍         | 565/11625 [00:17<08:08, 22.65it/s]








  5%|▍         | 570/11625 [00:17<06:57, 26.45it/s]








  5%|▍         | 575/11625 [00:17<06:34, 27.99it/s]








  5%|▍         | 579/11625 [00:18<06:00, 30.60it/s]








  5%|▌         | 583/11625 [00:18<05:44,

  8%|▊         | 949/11625 [00:30<04:05, 43.50it/s]








  8%|▊         | 955/11625 [00:30<03:46, 47.11it/s]








  8%|▊         | 962/11625 [00:30<03:38, 48.72it/s]








  8%|▊         | 968/11625 [00:30<06:01, 29.49it/s]








  8%|▊         | 974/11625 [00:30<05:06, 34.70it/s]








  8%|▊         | 979/11625 [00:30<05:41, 31.16it/s]








  8%|▊         | 984/11625 [00:31<05:12, 34.05it/s]








  9%|▊         | 989/11625 [00:31<05:10, 34.25it/s]








  9%|▊         | 993/11625 [00:31<06:07, 28.95it/s]








  9%|▊         | 999/11625 [00:31<05:29, 32.27it/s]








  9%|▊         | 1005/11625 [00:31<04:43, 37.42it/s]








  9%|▊         | 1010/11625 [00:31<04:57, 35.69it/s]








  9%|▊         | 1015/11625 [00:31<04:39, 37.95it/s]








  9%|▉         | 1020/11625 [00:32<04:34, 38.61it/s]








  9%|▉         | 1025/11625 [00:32<05:27, 32.32it/s]








  9%|▉         | 1029/11625 [00:32<05:24, 32.69it/s]








  9%|▉         | 1033/11625 [00:32

 12%|█▏        | 1418/11625 [00:45<04:25, 38.39it/s]








 12%|█▏        | 1423/11625 [00:45<05:04, 33.54it/s]








 12%|█▏        | 1427/11625 [00:45<06:22, 26.65it/s]








 12%|█▏        | 1431/11625 [00:45<07:46, 21.85it/s]








 12%|█▏        | 1435/11625 [00:46<07:20, 23.14it/s]








 12%|█▏        | 1438/11625 [00:46<06:54, 24.61it/s]








 12%|█▏        | 1442/11625 [00:46<06:11, 27.41it/s]








 12%|█▏        | 1446/11625 [00:46<05:52, 28.89it/s]








 12%|█▏        | 1450/11625 [00:46<06:42, 25.28it/s]








 12%|█▏        | 1453/11625 [00:46<07:54, 21.44it/s]








 13%|█▎        | 1457/11625 [00:46<06:51, 24.72it/s]








 13%|█▎        | 1460/11625 [00:47<08:27, 20.04it/s]








 13%|█▎        | 1464/11625 [00:47<07:41, 21.99it/s]








 13%|█▎        | 1467/11625 [00:47<07:14, 23.38it/s]








 13%|█▎        | 1472/11625 [00:47<06:17, 26.88it/s]








 13%|█▎        | 1482/11625 [00:47<04:57, 34.11it/s]








 13%|█▎        | 1487/11

 16%|█▌        | 1830/11625 [01:00<03:39, 44.52it/s]








 16%|█▌        | 1839/11625 [01:00<03:08, 51.97it/s]








 16%|█▌        | 1849/11625 [01:00<02:41, 60.55it/s]








 16%|█▌        | 1864/11625 [01:01<02:22, 68.31it/s]








 16%|█▌        | 1877/11625 [01:01<02:02, 79.32it/s]








 16%|█▌        | 1887/11625 [01:01<02:22, 68.21it/s]








 16%|█▋        | 1896/11625 [01:01<02:19, 69.65it/s]








 16%|█▋        | 1904/11625 [01:01<02:25, 66.98it/s]








 16%|█▋        | 1912/11625 [01:01<02:29, 64.97it/s]








 17%|█▋        | 1919/11625 [01:01<02:58, 54.40it/s]








 17%|█▋        | 1926/11625 [01:02<04:59, 32.41it/s]








 17%|█▋        | 1932/11625 [01:02<04:24, 36.59it/s]








 17%|█▋        | 1944/11625 [01:02<03:31, 45.76it/s]








 17%|█▋        | 1955/11625 [01:02<02:54, 55.42it/s]








 17%|█▋        | 1963/11625 [01:02<02:51, 56.19it/s]








 17%|█▋        | 1971/11625 [01:02<02:55, 55.05it/s]








 17%|█▋        | 1978/11

...

 81%|████████  | 9366/11625 [10:47:28<1781:23:15, 2838.86s/it]








 81%|████████  | 9370/11625 [10:47:28<1244:46:12, 1987.22s/it]








 81%|████████  | 9373/11625 [10:47:29<870:11:18, 1391.07s/it] 








 81%|████████  | 9377/11625 [10:47:29<608:03:17, 973.75s/it] 








 81%|████████  | 9381/11625 [10:47:29<424:53:14, 681.64s/it]








 81%|████████  | 9385/11625 [10:47:29<296:53:48, 477.16s/it]








 81%|████████  | 9388/11625 [10:47:29<207:33:20, 334.02s/it]








 81%|████████  | 9393/11625 [10:47:29<144:58:09, 233.82s/it]








 81%|████████  | 9398/11625 [10:47:29<101:15:19, 163.68s/it]








 81%|████████  | 9403/11625 [10:47:29<70:43:26, 114.58s/it] 








 81%|████████  | 9408/11625 [10:47:30<49:23:59, 80.22s/it] 








 81%|████████  | 9413/11625 [10:47:30<34:30:27, 56.16s/it]








 81%|████████  | 9418/11625 [10:47:30<24:06:19, 39.32s/it]








 81%|████████  | 9423/11625 [10:47:30<16:50:24, 27.53s/it]








 81%|████████  | 9428/11625 [10:47

 84%|████████▍ | 9806/11625 [10:47:40<00:55, 32.52it/s]








 84%|████████▍ | 9812/11625 [10:47:41<00:51, 34.98it/s]








 84%|████████▍ | 9816/11625 [10:47:41<00:49, 36.31it/s]








 84%|████████▍ | 9820/11625 [10:47:41<00:51, 35.13it/s]








 85%|████████▍ | 9824/11625 [10:47:41<00:51, 34.87it/s]








 85%|████████▍ | 9828/11625 [10:47:41<00:49, 36.26it/s]








 85%|████████▍ | 9833/11625 [10:47:41<00:47, 37.38it/s]








 85%|████████▍ | 9838/11625 [10:47:41<00:45, 39.67it/s]








 85%|████████▍ | 9843/11625 [10:47:41<00:47, 37.54it/s]








 85%|████████▍ | 9848/11625 [10:47:42<00:44, 39.94it/s]








 85%|████████▍ | 9853/11625 [10:47:42<00:44, 39.60it/s]








 85%|████████▍ | 9858/11625 [10:47:42<00:47, 37.14it/s]








 85%|████████▍ | 9862/11625 [10:47:42<00:46, 37.94it/s]








 85%|████████▍ | 9867/11625 [10:47:42<00:43, 40.71it/s]








 85%|████████▍ | 9872/11625 [10:47:42<00:46, 37.91it/s]








 85%|████████▍ | 9876/11625 [10:47:42<00

 88%|████████▊ | 10252/11625 [10:47:52<00:32, 42.84it/s]








 88%|████████▊ | 10257/11625 [10:47:52<00:33, 41.02it/s]








 88%|████████▊ | 10262/11625 [10:47:52<00:37, 36.54it/s]








 88%|████████▊ | 10267/11625 [10:47:52<00:38, 35.37it/s]








 88%|████████▊ | 10275/11625 [10:47:52<00:32, 41.91it/s]








 88%|████████▊ | 10281/11625 [10:47:52<00:34, 39.49it/s]








 88%|████████▊ | 10286/11625 [10:47:53<00:36, 36.81it/s]








 89%|████████▊ | 10295/11625 [10:47:53<00:30, 44.10it/s]








 89%|████████▊ | 10301/11625 [10:47:53<00:30, 43.17it/s]








 89%|████████▊ | 10306/11625 [10:47:53<00:30, 43.17it/s]








 89%|████████▊ | 10311/11625 [10:47:53<00:35, 36.82it/s]








 89%|████████▊ | 10316/11625 [10:47:53<00:40, 32.54it/s]








 89%|████████▉ | 10322/11625 [10:47:53<00:36, 35.68it/s]








 89%|████████▉ | 10326/11625 [10:47:54<00:37, 34.75it/s]








 89%|████████▉ | 10330/11625 [10:47:54<00:45, 28.45it/s]








 89%|████████▉ | 10334/11

 92%|█████████▏| 10733/11625 [10:48:04<00:29, 30.05it/s]








 92%|█████████▏| 10737/11625 [10:48:04<00:29, 30.13it/s]








 92%|█████████▏| 10741/11625 [10:48:04<00:28, 31.38it/s]








 92%|█████████▏| 10746/11625 [10:48:04<00:26, 32.75it/s]








 92%|█████████▏| 10751/11625 [10:48:05<00:25, 34.73it/s]








 93%|█████████▎| 10756/11625 [10:48:05<00:22, 38.21it/s]








 93%|█████████▎| 10761/11625 [10:48:05<00:24, 35.97it/s]








 93%|█████████▎| 10765/11625 [10:48:05<00:27, 31.81it/s]








 93%|█████████▎| 10769/11625 [10:48:05<00:30, 28.50it/s]








 93%|█████████▎| 10773/11625 [10:48:05<00:29, 28.93it/s]








 93%|█████████▎| 10777/11625 [10:48:05<00:26, 31.53it/s]








 93%|█████████▎| 10781/11625 [10:48:06<00:27, 30.33it/s]








 93%|█████████▎| 10786/11625 [10:48:06<00:24, 33.59it/s]








 93%|█████████▎| 10790/11625 [10:48:06<00:24, 33.85it/s]








 93%|█████████▎| 10794/11625 [10:48:06<00:25, 32.04it/s]








 93%|█████████▎| 10799/11

 96%|█████████▋| 11199/11625 [10:48:15<00:11, 36.93it/s]








 96%|█████████▋| 11205/11625 [10:48:16<00:10, 39.47it/s]








 96%|█████████▋| 11211/11625 [10:48:16<00:09, 41.98it/s]








 96%|█████████▋| 11216/11625 [10:48:16<00:11, 36.63it/s]








 97%|█████████▋| 11221/11625 [10:48:16<00:10, 37.73it/s]








 97%|█████████▋| 11226/11625 [10:48:16<00:10, 39.08it/s]








 97%|█████████▋| 11231/11625 [10:48:16<00:09, 40.18it/s]








 97%|█████████▋| 11236/11625 [10:48:16<00:09, 40.45it/s]








 97%|█████████▋| 11241/11625 [10:48:16<00:09, 41.98it/s]








 97%|█████████▋| 11246/11625 [10:48:17<00:10, 37.30it/s]








 97%|█████████▋| 11250/11625 [10:48:17<00:10, 37.23it/s]








 97%|█████████▋| 11257/11625 [10:48:17<00:08, 42.46it/s]








 97%|█████████▋| 11262/11625 [10:48:17<00:08, 40.91it/s]








 97%|█████████▋| 11268/11625 [10:48:17<00:07, 44.84it/s]








 97%|█████████▋| 11276/11625 [10:48:17<00:06, 51.08it/s]








 97%|█████████▋| 11282/11

Разделите коллекцию текстов на обучающую и тестовую части. 

In [104]:
# for i in range(len(tops)):
#     for el in tops[i].most_common(5):
#         print(el)
#     print("\n")

Counter({('МВД', 'mospolice'): 4947, ('России', 'mospolice'): 3890, ('МЧС', 'mchsgov'): 3486, ('полиции', 'mospolice'): 2977, ('Москве', 'mospolice'): 2674, ('полиция', 'mospolice'): 2616, ('России', 'russianpost'): 2535, ('Минобороны', 'mil'): 1813, ('УВД', 'mospolice'): 1669, ('России', 'mchsgov'): 1588, ('новости', 'mospolice'): 1511, ('МЧСРОССИИ', 'mchsgov'): 1226, ('России', 'mil'): 1179, ('Москвы', 'mospolice'): 1168, ('Почты', 'russianpost'): 1130, ('года', 'mospolice'): 1114, ('ПочтаРоссии', 'russianpost'): 936, ('службы', 'mospolice'): 896, ('Сотрудники', 'mospolice'): 855, ('Почта', 'russianpost'): 826, ('обороны', 'mil'): 799, ('дел', 'mospolice'): 791, ('военного', 'mil'): 736, ('мвдроссии', 'mospolice'): 727, ('сотрудники', 'mospolice'): 711, ('МВДРоссии', 'mospolice'): 705, ('ходе', 'mil'): 697, ('полицияроссии', 'mospolice'): 690, ('сотрудников', 'mospolice'): 687, ('РФ', 'mospolice'): 681, ('года', 'russianpost'): 678, ('начальника', 'mospolice'): 664, ('району', 'mospo

 ## Задание 2 [2 балла]. Классификация по правилам
 
 * Разработайте несколько правил вида "Если встречается слово $w$, то текст относится к паблику $c$"
 * Посчитайте, какую точность, полноту, $f$-меру и $accuracy$ вы получаете при классификации по правилам
 * Получилось ли у вас придумать правило, которое никогда не ошибается?

## Задание 3 [3 балла]. Baseline
Используйте стандартный ```sklearn.pipeline``` для классификации текстов: 
* векторизация 
* $tf-idf$ взвешивание 
* ваш любимый метод классификации.


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Задание 4 [2 балла]. Снижение размерности
Добавьте в ваш ```sklearn.pipeline```  методы снижения размерности:  PCA / LSI / LSA / LDA / другое. Какие методы классификации разумно использовать после снижения размерности? Как изменились результаты классификации после добавления нового шага?

## Задание 5 [1 балл]. Лемматизация
Посмотрите, как влияет лемматизация на качество классификации. Как изменится качество классификации, если вы используете ```CountVectorizer``` на словах или $n$-граммах на лемматизированных текстах?

## Задание 6 [3 балла]. Continious bag of words
Для baseline решения мы использовали обычное представление текста в виде мешка слов. Попробуйте использовать другие модели представления текста – например, в виде непрерывного мешка слов, то есть, в виде набора эмбеддингов. Для того, чтобы получить вектор текста попробуйте:
* усреднить все эмбеддинги слов, входящих в этот текст
* усреднить все эмбеддинги слов, входящих в этот текст с $tf-idf$ весами
* использовать любую модель эмбеддинга документа.

Используйте любую модель эмбеддингов по вашему вкусу. 


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Задание 7 [2 балла]. fastText

Используйте ```fastText``` в режиме классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Задание 8 [4 балла]. CNN

Реализуйте модель Kim et al (2014) для решения задачи классификации с помощью CNN. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).
Ссылка: Kim Y. Convolutional Neural Networks for Sentence Classification. 2014

## Задание 9 [4 + 2 балла]. RNN

(4 балла)Используйте ```RNN``` (BLSTM с какими-то признаками и пулинг поверх) для решения задачи текстовой классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).

За дополнительные 2 балла добавьте в модель символьные признаки - CharCNN или CharRNN

## Задание 10 [8 баллов]. ULMFit

Используйте ```ULMFit``` для решения задачи классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Конец
Выполните какие-то из предыдущих заданий. Для всех заданий, кроме задания 1 требуется вычислить метрику accuracy метода.

Подведите итоги и проведите сравнение всех использованных методов. Какой из них показался вам лучше и почему?

**NB!** Задание обязательное вне зависимости от того, сколько из предыдущих пунктов вы выполнили, и дополнительных баллов не дает.


Для получения полной оценки за NLP-часть достаточно набрать **20 баллов**.

# Правила сдачи 

1. Домашняя работа должна быть выполнена в ipynb-тетрадке.
2. Сделанную тетрадку нужно отправить ассистенту (ссылка на контакты будет в вики).
3. Задание выполняется индивидуально.
4. Все вычисления должны быть снабжены пояснениями!
5. Дедлайн – 10 июня в 10.00.
